In [1]:
# library imports
# If this fails, check if you are running
# from the active "llms" environment on the command line
import os
import re
import requests
import socket
from requests.exceptions import ConnectionError
from requests.exceptions import MissingSchema
from requests.exceptions import InvalidSchema
from urllib3.exceptions import MaxRetryError, NameResolutionError
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
import ollama

In [2]:
# load_dotenv - ładuje wszystkie zmienne do środowiska
load_dotenv(override=True)
# nazwa używanego modelu LLM
MODEL = 'wizardlm2:7b'

In [3]:
# Niektóre strony internetowe wymagają użycia odpowiednich nagłówków podczas ich pobierania:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
(KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# Klasa reprezentująca zescrapowaną stronę internetową z linkami
class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [130]:
#page = Website("https://huggingface.co")
page = Website("https://pl.wikipedia.org/wiki/Wikipedia:Strona_główna")
# sprawdźmy jakie linki znajdziemy na stronie
page.links

['#bodyContent',
 '/wiki/Wikipedia:Strona_g%C5%82%C3%B3wna',
 '/wiki/Specjalna:Losowa_strona',
 '/wiki/Portal:Kategorie_G%C5%82%C3%B3wne',
 '/wiki/Wikipedia:Wyr%C3%B3%C5%BCniona_zawarto%C5%9B%C4%87_Wikipedii',
 '/wiki/Pomoc:FAQ',
 '/wiki/Wikipedia:O_Wikipedii',
 '/wiki/Wikipedia:Kontakt_z_wikipedystami',
 '/wiki/Pomoc:Pierwsze_kroki',
 '/wiki/Wikipedia:Portal_wikipedyst%C3%B3w',
 '/wiki/Wikipedia:Tablica_og%C5%82osze%C5%84',
 '/wiki/Wikipedia:Zasady',
 '/wiki/Pomoc:Spis_tre%C5%9Bci',
 '/wiki/Specjalna:Ostatnie_zmiany',
 '/wiki/Specjalna:Strony_specjalne',
 '/wiki/Wikipedia:Strona_g%C5%82%C3%B3wna',
 '/wiki/Specjalna:Szukaj',
 'https://donate.wikimedia.org/?wmf_source=donate&wmf_medium=sidebar&wmf_campaign=pl.wikipedia.org&uselang=pl',
 '/w/index.php?title=Specjalna:Utw%C3%B3rz_konto&returnto=Wikipedia%3AStrona+g%C5%82%C3%B3wna',
 '/w/index.php?title=Specjalna:Zaloguj&returnto=Wikipedia%3AStrona+g%C5%82%C3%B3wna',
 'https://donate.wikimedia.org/?wmf_source=donate&wmf_medium=sidebar&wmf_

In [4]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in \
a brochure about the company, such as links to an About page, or a Company page, \
or Careers/Jobs pages.\n"
link_system_prompt += "You should respond only in JSON, without text, object as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [132]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond only in JSON, without text, object as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for \
a brochure about the company, respond with the full https URL in clean JSON format \
wihout text json on the beginning of the response. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [134]:
print(get_links_user_prompt(page))

Here is the list of links on the website of https://pl.wikipedia.org/wiki/Wikipedia:Strona_główna - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in clean JSON format wihout text json on the beginning of the response. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#bodyContent
/wiki/Wikipedia:Strona_g%C5%82%C3%B3wna
/wiki/Specjalna:Losowa_strona
/wiki/Portal:Kategorie_G%C5%82%C3%B3wne
/wiki/Wikipedia:Wyr%C3%B3%C5%BCniona_zawarto%C5%9B%C4%87_Wikipedii
/wiki/Pomoc:FAQ
/wiki/Wikipedia:O_Wikipedii
/wiki/Wikipedia:Kontakt_z_wikipedystami
/wiki/Pomoc:Pierwsze_kroki
/wiki/Wikipedia:Portal_wikipedyst%C3%B3w
/wiki/Wikipedia:Tablica_og%C5%82osze%C5%84
/wiki/Wikipedia:Zasady
/wiki/Pomoc:Spis_tre%C5%9Bci
/wiki/Specjalna:Ostatnie_zmiany
/wiki/Specjalna:Strony_specjalne
/wiki/Wikipedia:Strona_g%C5%82%C3%B3wna
/wiki/Specjalna:Szukaj
https://donate.wikimedia.org/?wmf_source=donate&wmf_

In [6]:
def get_links(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        options={"format": "json"}
    )
    result = response['message']['content']
    
    # Odkomentuj i użyj tych linii, aby oczyścić odpowiedź LLM.
    # To znacznie zwiększa szansę na poprawne sparsowanie JSON.
    result = re.sub(r'<think>.*?</think>', '', result, flags=re.DOTALL)
    result = re.sub(r'```json\n|\n```$', "", result.strip(), flags=re.MULTILINE).strip()
    
    print("--- Raw LLM output after cleaning ---")
    print(result)
    print("------------------------------------")

    try:
        content_json = json.loads(result)
        return content_json
    except json.JSONDecodeError:
        print("Odpowiedź nie jest poprawnym JSON. Zwracam pustą listę linków.")
        # Zwróć pustą, ale poprawną strukturę, aby uniknąć błędu 'NoneType'.
        return {"links": []}

In [136]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-R1-0528',
 '/google/gemma-3n-E4B-it-litert-preview',
 '/ResembleAI/chatterbox',
 '/Qwen/Qwen3-Embedding-0.6B-GGUF',
 '/deepseek-ai/DeepSeek-R1-0528-Qwen3-8B',
 '/models',
 '/spaces/ResembleAI/Chatterbox',
 '/spaces/enzostvs/deepsite',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/alexnasa/Chain-of-Zoom',
 '/spaces/NihalGazi/Text-To-Speech-Unlimited',
 '/spaces',
 '/datasets/yandex/yambda',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/open-thoughts/OpenThoughts3-1.2M',
 '/datasets/open-r1/Mixture-of-Thoughts',
 '/datasets/Hcompany/WebClick',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Wri

In [ ]:
#get_links("https://huggingface.co")
get_links("https://pl.wikipedia.org/wiki/Wikipedia:Strona_główna")

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links_data = get_links(url)

    # Sprawdź, czy klucz "links" istnieje i nie jest pusty
    if links_data and "links" in links_data:
        for link_item in links_data["links"]:
            try:
                # Sprawdzamy, czy element listy jest słownikiem (oczekiwany format)
                if isinstance(link_item, dict):
                    page_url = link_item.get("url")
                    page_type = link_item.get("type", "Link") # Użyj "Link" jako domyślnego typu
                    if not page_url:
                        continue # Pomiń, jeśli nie ma URL
                    
                    result += f"\n\n--- {page_type} ---\n"
                    result += Website(page_url).get_contents()
                
                # Sprawdzamy, czy element listy jest stringiem (format awaryjny)
                elif isinstance(link_item, str):
                    page_url = link_item
                    result += f"\n\n--- Link: {page_url} ---\n"
                    result += Website(page_url).get_contents()

            except socket.gaierror as e:
                print(f"DNS resolution failed for {page_url}: {e}")
            except NameResolutionError as e:
                print(f"Name resolution error for {page_url}: {e}")
            except MaxRetryError as e:
                print(f"Max retries exceeded for {page_url}: {e}")
            except ConnectionError as e:
                print(f"Connection error for {page_url}: {e}")
            except MissingSchema as e:
                print(f"Invalid URL schema for {page_url}: {e}")
            except InvalidSchema as e:
                print(f"Omitted unsupported URL for {page_url} (InvalidSchema): {e}")
            except Exception as e:
                print(f"An unexpected error occurred for {page_url}: {e}")

    return result

In [ ]:
# mogą się zdarzyć linki, które nie działają poprawnie
# wyrzuci wtedy błąd połączenia i teoretycznie powinno pomóc
# ponowne uruchomienie niniejszego bloku kodu lub użycie lepszego modelu
# w razie czego w definicji funkcji mamy wyjątki to obsługujące
#print(get_all_details("https://huggingface.co"))
print(get_all_details("https://pl.wikipedia.org/wiki/Wikipedia:Strona_główna"))

In [ ]:
#system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
#and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
#Include details of company culture, customers and careers/jobs if you have the information."

# Lub użyj wersji bardziej humorystycznej testując łatwość zmiany "tonu" naszych wyników. Wystarczy odkomentować

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. \
Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; \
use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Obcięcie, jeśli więcej niż 5000 znaków
    return user_prompt

In [ ]:
#get_brochure_user_prompt("HuggingFace", "https://huggingface.co")
#get_brochure_user_prompt("Wikipedia", "https://pl.wikipedia.org/wiki/Wikipedia:Strona_główna")
get_brochure_user_prompt("Booking", "https://www.booking.com/index.pl.html")

In [114]:
def create_brochure(company_name, url):
    response = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response['message']['content']
    # w razie czego usuwamy tekst wnioskowania z modeli typu reasoning
    # odkomentuj w razie potrzeby:
    result = re.sub(r'<think>.*?</think>', '', result, flags=re.DOTALL)
    result = result.strip()
    display(Markdown(result))

In [115]:
#create_brochure("HuggingFace", "https://huggingface.co")
create_brochure("Wikipedia", "https://pl.wikipedia.org/wiki/Wikipedia:Strona_główna")
#create_brochure("HuggingFace", "https://huggingface.co")

--- Raw LLM output after cleaning ---
Oto wykaz stron internetowych, które są załącznikiem do strony Wikipedia:Strona_główna:

1. https://commons.wikimedia.org/wiki/File:James_Tooley,_Jr._-_Portrait_of_Andrew_Jackson_(1840)_-_Google_Art_Project.jpg
2. https://commons.wikimedia.org/wiki/File:Tim_Berners-Lee.jpg
3. https://creativecommons.org/licenses/by-sa/4.0/deed.pl
4. https://foundation.wikimedia.org/wiki/Policy:Terms_of_Use/pl
5. https://foundation.wikimedia.org/wiki/Special:MyLanguage/Policy:Privacy_policy
6. https://www.wikidata.org/wiki/Wikidata:Strona_główna
7. https://pl.wikinews.org/wiki/
8. https://species.wikimedia.org/wiki/Strona_główna
9. https://pl.wikibooks.org/wiki/
10. https://pl.wikiquote.org/wiki/
11. https://pl.wikivoyage.org/wiki/
12. https://beta.wikiversity.org/wiki/Strona_Główna
13. https://www.wikifunctions.org/wiki/
14. https://wikimediafoundation.org/
15. https://wikimedia.pl
16. https://meta.wikimedia.org/wiki/Strona_główna

Oto wykaz stron, które są załączn

# Wikpedia - Wolna Encyklopedia dla Wszystkich

## O Nas

Cześć! Witamy na Wikipedii - wolnej encyklopedii, którą każdy może redagować. Z naszymi 1,6 milionami artykułów i 60 językami, jesteś w dowolnym miejscu świata.

## Nasza Kultura

W Wikipedii żyje się wolnością i otwartością. Każdy może editować naszą encyklopedię, niezależnie od wieku, narodowości lub pozycji społecznej. To jest nasza siła i nasze wyzwanie.

## Czym Zasłużymy Państwo?

Nasza encyklopedia jest źródłem wiedzy dla wszystkich. Utwórz konto i zacznij redagować. Wyświetl nowy artykuł, popraw jakiś błąd lub proponuj nową ideę. W Wikipedii, wolna encyklopedia dla wszystkich.

### Statystyki

- **1 660 164** - liczba artykułów
- **5224** - ilość wyróżnionych artykułów
- **60** - liczba języków

## Wydarzenia

Więcej... [Wyświetl nowe wydarzenie](https://pl.wikipedia.org/Portal:Wydarzenia)

### Konkursy

Wygrał wybory prezydenckie w Korei Południowej. Więcej... [Dowiedz się więcej](https://pl.wikipedia.org/Special:RecentChanges)

## O Naszych Zmarłych

Ostatnio zmarli: Vladimír Smutný, Ewa Dałkowska, Edgar Lungu. Więcej... [Zobacz cały list](https://pl.wikipedia.org/Portal:Zmarli)

## Rocznice

8 czerwca - imieniny Jadwigi, Medarda i Seweryna. Więcej... [Więcej informacji](https://pl.wikipedia.org/Portal:Rocznice)

In [74]:
def stream_brochure(company_name, url):
    stream = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )
    result = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        result += chunk['message']['content'] or ''
        #usuwamy tekst wnioskowania z modeli typu reasoning
        # odkomentuj w razie potrzeby:
        result = re.sub(r'<think>.*?</think>', '', result, flags=re.DOTALL)
        result = result.strip()
        result = result.replace("```", "").replace("markdown", "")
        update_display(Markdown(result), display_id=display_handle.display_id)

In [138]:
#stream_brochure("HuggingFace", "https://huggingface.co")
stream_brochure("Booking", "https://www.booking.com/index.pl.html")

--- Raw LLM output after cleaning ---
It appears you've provided a list of URLs associated with Booking.com, a global online travel agency that specializes in holiday rentals and hotel reservations. These URLs cover various aspects of the platform, including user authentication, account management, property reviews, language-specific pages, and general website navigation.

Some of these links are specific to single sign-on (SSO) integration with various services like Google, Apple, Facebook, and others, indicating that users can log in or register using their existing accounts from these providers.

The URLs also include references to session information, timestamps, and state parameters, which are typically used to maintain user sessions and ensure security during the authentication process.

If you have a specific question or task related to these URLs or the services they represent, feel free to clarify, and I'll be happy to assist you further!
------------------------------------
O

🎉 **Witaj w Świecie Możliwości z Booking.com!** 🌎✨

Czy pan/pani gotowy jest na niezwykłą podróż? Nie musisz spuszczać ani palec, ponieważ Booking.com to twoja kluczowa kartka do odkrycia wielu światów! Czy chcemy zazaścić przystanek w Warszawie, rozwalić rytuały w Krakowie czy odczymiać czar Rosyjskiej Finy? Oto tylko kilka kliknięć od niezwykłego doświadczenia!

**O nas:**
Booking.com to Twój super-przeglądarka internetowa, która z życznym uśmiechem otwiera drzwi do miliona obiektów na całym globusie. Od klasycznych hoteli po unikalne apartamenty, od domków letniskowych po luksusowe wille – mamy co zadowoli każdego podróżnika!

**Kultura Booking.com:**
W naszej firmie panuje duch klienta. To oni są gwiazdą naszego show, a my tylko zapraszamy go na najlepszy pokaz! Jesteśmy zespołem kreatywnych, odważnych i pasjonatów podróży, którzy każde dzień pracują nad tym, aby uczynić Twoją podróż nie tylko pożycią, ale rzeczą, która zostanie pamięcią na całe życie.

**Naszy Klient:**
Twoje memu satysfakcji? Skończyło się wieczorem w domku nad jeziorem? Otrzymałeś upgrade na srebrny stanu w luksusowym hotelu w Monte Carlo? Podziel się tym z nami! Nasze media społecznościowe płyną z historii, które pokazują, że nasze serwisy nie są tylko słowem – to rzeczywistość!

**Możliwości Kariery:**
Czy chciałbyś dołączyć do naszej podróży? W Booking.com każdy z nas ma szansę rosnąć, eksperymentować i wpływać na to, jak postrzegają świat podróże miliony osób. Oferujemy nie tylko pozycje z graneń, ale również miejsca, gdzie można kształcić się i rozwijać karierę w dynamicznym, międzynarodowym środowisku.

**Dlaczego Booking.com?**
Prosto mówiąc – to nasza pasja! Każdy z nas zakładamy, że podróż jest nie tylko połączeniem punktów A i B, ale odkryciem nowych horyzontów, kultur i sposobów na życie. Dlatego staramy się stworzyć doświadczenia, które przekręcają oczekiwania naszych klientów.

**Dlaczego wybierać Booking.com?**
- **Oferty specjalne:** Zdobyj punktów w programie lojalnościowego Genius i cieszy się promocjami sezonowymi!
- **Wybór:** Od taniego zakwaterowania po luksusowe wycieczki – mamy opcję, która odpowiada Twoim potrzebom.
- **Komfort:** Zanurz się w pragnienie kąpieli po długim dniu eksploracji lub skorzystaj z parkingu, jeśli przybywasz samochodem.
- **Dostępność:** Czy tatusiek zadaje pytanie? Nasza obsługa klienta działa 24/7, aby pomóc w każdej sytuacji!
- **Wsparcie:** Z nami na dobre wykonanych wyjazdach. Skontaktuj się z nami za każdym razem, gdy masz pytania lub potrzebujesz pomocy.

**Przyłącz się do nas!**
Oto tylko kilka kroków od tego, aby zostać częścią naszej podróży:
1. Zarejestruj się na stronie Booking.com.
2. Przeczytaj o programie lojalnościowym Genius i zacznij zarabianac z każdej rezerwacji.
3. Sprawdź dostępne pozycje pracy i zainteresuj się możliwościami kariery w naszej ekipie.

**Booking.com – Twoja eksperymentalna laboratorium podróży!** 🚀🌟

---

*Chętnie przyjmujemy krytykę i sugestie, aby stworzyć jeszcze bardziej zabawną i angażującą broszurę. Niezwykłe podziękowania za twoją kreatywność i współpracę!* 🤝😄